In [ ]:
#Compute limits
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Sep 22 06:09:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!nvidia-smi
#!kill process_id

Sun Sep 22 06:09:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install transformers
!pip install accelerate
!pip install datasets
!pip install rouge_score
!pip install peft
!pip install trl
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for r

In [ ]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import statistics

# **Load datasets and merge them**

In [ ]:
!wget -O Dataset-1.zip https://github.com/ch22group16/capstone-GenAI/raw/main/dataset/Dataset-1-20240909T162549Z-001.zip
!wget -O Dataset-2.zip https://github.com/ch22group16/capstone-GenAI/raw/main/dataset/Dataset-2-20240909T162549Z-001.zip
%rm -rf /content/Dataset-1/ /content/Dataset-2/

import zipfile
import os

def extract_zip(zip_path, extract_to):
    # Ensure the extraction directory exists
    os.makedirs(extract_to, exist_ok=True)

    # Open the ZIP file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Extract all the contents to the specified directory
        zip_ref.extractall(extract_to)

zip_path = ['/content/Dataset-1.zip', '/content/Dataset-2.zip']
extract_to = '/content'
for path in zip_path:
  extract_zip(path, extract_to)

# Load the Excel files into DataFrames
dftr = pd.concat([pd.read_excel('/content/Dataset-1/aiml-qa-train.xlsx'),
                  pd.read_csv('/content/Dataset-2/train.csv'),
                  pd.read_csv('/content/Dataset-2/dev.csv'),
                  pd.read_excel('/content/Dataset-1/aiml-qa-dev.xlsx', usecols=[0,1], names=['question', 'answer']),
                  pd.read_excel('/content/Dataset-1/aiml-qa-dev.xlsx', usecols=[0,2], names=['question', 'answer'])])
dfts1 = pd.read_excel("/content/Dataset-1/aiml-qa-test.xlsx")
dfts2 = pd.read_csv("/content/Dataset-2/test.csv")

--2024-09-22 06:11:33--  https://github.com/ch22group16/capstone-GenAI/raw/main/dataset/Dataset-1-20240909T162549Z-001.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ch22group16/capstone-GenAI/main/dataset/Dataset-1-20240909T162549Z-001.zip [following]
--2024-09-22 06:11:34--  https://raw.githubusercontent.com/ch22group16/capstone-GenAI/main/dataset/Dataset-1-20240909T162549Z-001.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 150066 (147K) [application/zip]
Saving to: ‘Dataset-1.zip’

Dataset-1.zip       100%[===================>] 146.55K  --.-KB/s    in 0.03s   

2024-09-22 06:11:34 (

In [ ]:
dftr.head()

,question,answer
0,What is the difference between concatenation v...,Concatenation combines two tensors by adding t...
1,What is the difference between concatenation v...,Concatenation is often used to combine differe...
2,Why are derivatives substracted from weights?,The derivative of the loss function at a point...
3,Why are derivatives substracted from weights?,The intuition behind adjusting the weights by ...
4,Describe a process/pipeline for generating rep...,Select a pretrained model suitable for the tas...


In [ ]:
dfts1.head()

,question,answer1,answer2
0,How we can effectively convert 2D images to 1D?,Converting images to 1D data may not be effect...,"To effectively convert 2D images to 1D, use te..."
1,Can we utilize an autoencoder to perform dimen...,"Yes, autoencoders can be applied to numerical ...","Yes, autoencoders can be used for dimensionali..."
2,What is NLP's current biggest challenge that i...,The main challenges of NLP is finding and coll...,NLP models struggle with tasks that require re...
3,Which problems cannot be solved by Neural netw...,While neural networks have shown great success...,"Neural networks are powerful, but they may str..."
4,Is scaling necessary for SVM?,"Yes, scaling the input data is generally recom...",Scaling the input data is advisable when utili...


In [ ]:
dfts2.head()

,question,answer
0,What is feature selection?,The process of selecting a subset of relevant ...
1,"What are Markov decision processes (MDP), and ...",MDPs are mathematical frameworks for modeling ...
2,What is batch normalization and how does it work?,Normalizes the layer's input over a mini-batch
3,How does Word2Vec create word embeddings?,Word2Vec creates word embeddings by training o...
4,What is the role of hidden layers in MLP?,Hidden layers in MLP help capture complex patt...


# **Load model**

In [ ]:
import os
import transformers
import torch
from datasets import load_dataset, Dataset, DatasetDict
from trl import SFTTrainer
from peft import LoraConfig, PeftModel, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

# Download Gemma 2b-it base model
model_id = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config = bnb_config,
                                             device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
# Configure LoRA
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM"
)


In [ ]:
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

1,515,268,096 total parameters.
524,363,776 training parameters.


In [ ]:
# ZERO SHOT TESTING
device = "cuda"

dfts2['generated_answer'] = None
def generate_answer(question):
    input_ids = tokenizer.encode("Question: " + question + "\nAnswer:", return_tensors='pt').to(device)
    attention_mask = torch.ones(input_ids.shape, device=device)
    output = model.generate(input_ids, attention_mask = attention_mask, max_new_tokens=60)
    gen_text = tokenizer.decode(output[0], skip_special_tokens=True)
    question, answer = gen_text.split('\nAnswer:', maxsplit=1)
    return answer

dfts2['generated_answer'] = dfts2['question'][:5].apply(generate_answer)
for index, row in dfts2[:3].iterrows():
  print(f'Question: {row.iloc[0]} \nGiven_Answer: {row.iloc[1]} \nGenerated_Answer: {row.iloc[2]} \n')

Question: What is feature selection? 
Given_Answer: The process of selecting a subset of relevant features to improve model performance and reduce complexity. 
Generated_Answer:  Feature selection is a process of identifying and selecting the most relevant features for a given machine learning task. This process aims to improve the performance of the model by reducing the dimensionality of the feature space and selecting features that are most likely to contribute to the target variable. 

Question: What are Markov decision processes (MDP), and how are they related to reinforcement learning? 
Given_Answer: MDPs are mathematical frameworks for modeling decision-making problems where outcomes are partly random and partly under the control of a decision-maker, providing the theoretical foundation for reinforcement learning algorithms. 
Generated_Answer:  Sure, here's a detailed explanation of Markov decision processes (MDPs) and how they relate to reinforcement learning:

**Markov Decisio

In [ ]:
## The Answers generated on zero shot are so good, that it's interesting to calculate rouge scores without finetuning the model.
dfts2['generated_answer'] = None
def generate_answer(question):
    input_ids = tokenizer.encode("Question: " + question + "\nAnswer:", return_tensors='pt').to(device)
    attention_mask = torch.ones(input_ids.shape, device=device)
    output = model.generate(input_ids, attention_mask = attention_mask, max_new_tokens=60)
    gen_text = tokenizer.decode(output[0], skip_special_tokens=True)
    question, answer = gen_text.split('\nAnswer:', maxsplit=1)
    return answer

dfts2['generated_answer'] = dfts2['question'].apply(generate_answer)


In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []
for index, row in dfts2.iterrows():
    target_answer = row['answer']
    generated_answer = row['generated_answer']
    scores = scorer.score(target_answer, generated_answer)
    rouge_scores.append(scores)

# Print average ROUGE scores
avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

print(f"Average ROUGE-1 Score: {avg_rouge1}")
print(f"Average ROUGE-2 Score: {avg_rouge2}")
print(f"Average ROUGE-L Score: {avg_rougeL}")

Average ROUGE-1 Score: 0.33956262483527266
Average ROUGE-2 Score: 0.1368003926694628
Average ROUGE-L Score: 0.2612245220352692


# **Train model**

In [ ]:
# Tokenize the Dataset

import json
train_data = []
for index, row in dftr.iterrows():
  train_data += [{'question': row.iloc[0], 'answer' : row.iloc[1]}]
with open('train_file.json', 'w') as file:
    json.dump(train_data, file)

from datasets import load_dataset
dataset = load_dataset('json', data_files = {'train':'train_file.json'})
tokenizer.pad_token = tokenizer.eos_token
def preprocess_function(examples):
    inputs = ["Question: " + q + "\nAnswer:" + a + tokenizer.eos_token for q, a in zip(examples['question'], examples['answer'])]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    labels = model_inputs['input_ids'].copy()
    model_inputs['labels'] = labels
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/3709 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Training arguments configuration
training_args = TrainingArguments(
    output_dir="./results",
    #evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir='./logs',
    logging_strategy='epoch',
    #logging_steps = 100,
    save_strategy = 'epoch',
    fp16=True
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    #eval_dataset=tokenized_dataset,  # Ideally, separate eval dataset
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss
100,0.444000
200,0.309500
300,0.314900
400,0.306500
500,0.296100
600,0.303100
700,0.284400
800,0.281700
900,0.292000


TrainOutput(global_step=928, training_loss=0.3136972039937973, metrics={'train_runtime': 897.6629, 'train_samples_per_second': 4.132, 'train_steps_per_second': 1.034, 'total_flos': 1.134670709784576e+16, 'train_loss': 0.3136972039937973, 'epoch': 1.0})

Test with sample queries

# **Evaluate model**

In [ ]:
model.eval()

dfts2['generated_answer'] = None
def generate_answer(question):
    input_ids = tokenizer.encode("Question: " + question + "\nAnswer:", return_tensors='pt').to(device)
    attention_mask = torch.ones(input_ids.shape, device=device)
    output = model.generate(input_ids, attention_mask = attention_mask, max_new_tokens=60)
    gen_text = tokenizer.decode(output[0], skip_special_tokens=True)
    question, answer = gen_text.split('\nAnswer:', maxsplit=1)
    return answer

dfts2['generated_answer'] = dfts2['question'].apply(generate_answer)

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []
for index, row in dfts2.iterrows():
    target_answer = row['answer']
    generated_answer = row['generated_answer']
    scores = scorer.score(target_answer, generated_answer)
    rouge_scores.append(scores)

# Print average ROUGE scores
avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

print(f"Average ROUGE-1 Score: {avg_rouge1}")
print(f"Average ROUGE-2 Score: {avg_rouge2}")
print(f"Average ROUGE-L Score: {avg_rougeL}")

Average ROUGE-1 Score: 0.45795621220112215
Average ROUGE-2 Score: 0.23268407878216557
Average ROUGE-L Score: 0.3846422578449109
